## SLF4J
SLF4J serves as abstraction for various logging frameworks, such as `java.util.logging`, `logback` and `reload4j`. It allows the end-user to plug in the desired logging framework at deployment time.  

As a library owner we need to just add SLF4J dependency (slf4j-api) and the other application which imports our library will have to add in an implementation. If there is no implementation, SLF4J will default to a no-operation implementation.

### Basic Usage
We need to first add `slf4j-api`:
```xml
<dependency>
    <groupId>org.slf4j</groupId>
    <artifactId>slf4j-api</artifactId>
    <version>2.0.6</version>
</dependency>
```

Next, we can add in the log lines

In [ ]:
import org.slf4j.Logger;
import org.slf4j.LoggerFactory;

public class HelloWorld {

  public static void main(String[] args) {
    Logger logger = LoggerFactory.getLogger(HelloWorld.class);
    logger.info("Hello World");
  }
}

Running the above code would print:
```
SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
```

However, if we add an implementation (also known as provider) such as `slf4j-simple`
```xml
<dependency>
    <groupId>org.slf4j</groupId>
    <artifactId>slf4j-simple</artifactId>
    <version>2.0.6</version>
    <scope>test</scope>
</dependency>
```
we would see the expected log line:
```
[main] INFO HelloWorld - Hello World
```

## SLF4J Providers
Some common SLF4J providers:
- Log4j 1.x:
```xml
<dependency>
    <groupId>org.slf4j</groupId>
    <artifactId>slf4j-log4j12</artifactId>
    <version>2.0.6</version>
</dependency>

```
- JDK 1.4 Logging:
```xml
<dependency>
    <groupId>org.slf4j</groupId>
    <artifactId>slf4j-jdk14</artifactId>
    <version>2.0.6</version>
</dependency>
```
- JCL:
```xml
<dependency>
    <groupId>org.slf4j</groupId>
    <artifactId>slf4j-jcl</artifactId>
    <version>1.7.36</version>
</dependency>
```

To change the logging framework, we just need to update the provider in classpath. `Logback` was creating keeping `SLF4J` in mind, therefor it doesn't need any adaptation layer. The following diagram explains the current scenario:  
![SLF4J](images/SLF4J_Providers.png)  

In short, as a library developer, we avoid imposing a logging framework on their end-user. The end-user may choose the desired logging framework at deployment time by inserting the corresponding slf4j binding on the classpath, which may be changed later by replacing an existing binding with another on the class path and restarting the application.  
Libraries or frameworks should not declare a dependency on any `SLF4J` binding/provider but only depend on `slf4j-api`. When a library declares a transitive dependency on a specific binding, that binding is imposed on the end-user negating the purpose of `SLF4J`.

### Version Matching
Mixing different versions of slf4j-api.jar and SLF4J binding can cause problems. If we are using `slf4j-api-2.0.6.jar`, then we should also use `slf4j-simple-2.0.6.jar`, using `slf4j-simple-1.5.5.jar` will not work. So consider the case where we depend upong two libraries - `BoneCP` which requires `slf4j-api` version 1.5 and `Hibernate` which requires `slf4j-api` version 1.6. How to use the correct binding? We need to update dependencies as:
```xml
<dependency>
  <groupId>org.slf4j</groupId>
  <artifactId>slf4j-api</artifactId>
  <version>1.7.25</version>
</dependency>   

<dependency>
  <groupId>org.slf4j</groupId>
  <artifactId>slf4j-log4j12</artifactId>
  <version>1.7.25</version>
</dependency>  
```

## Bridging legacy APIs
We may have several dependencies which do not make use of `slf4j-api`, instead they might be using `log4j`, `JCL` or `java.util.logging`. In such cases, we may use a bridging module which redirects calls made to `log4j`, `JCL` or `java.util.logging` APIs to behave as if they were made to the `SLF4J` API instead. This is how the flow looks:

![Legacy](images/legacy.png)

Note that we need to replace the original logging dependency with the briding ones mentioned above. For example, if a dependency X uses `JCL`, we need to replace ` commons-logging` dependency with `jcl-over-slf4j`. Similarly, if another dependency Y uses `Log4j`, we need to replace `log4j` with ` log4j-over-slf4j`.

Also note that the following combinations must not be used (it will cause infinite loop):
- `jcl-over-slf4j` and `slf4j-jcl` cannot be deployed at the same time
- `log4j-over-slf4j` and `slf4j-reload4j` cannot be present simultaneously
- `jul-to-slf4j` and `slf4j-jdk14` cannot be present simultaneously 

## Logback
Logback-classic requires the presence of `slf4j-api` and `logback-core` in addition to `logback-classic` on the classpath. Logback-access is another module in the Logback distribution which integrates with Servlet containers such as `Jetty` or `Tomcat` to provide rich and powerful HTTP-access log functionality. Here we will focus on Logback-classic.

### Architecture
Logback is built upon three main classes: `Logger`, `Appender` and `Layout`. The `Logger` class is part of the `logback-classic` module. On the other hand, the `Appender` and `Layout` interfaces are part of `logback-core`.  

**Loggers** are named entities. Their names are case-sensitive and they follow the hierarchical naming rule: *A logger is said to be an ancestor of another logger if its name followed by a dot is a prefix of the descendant logger name. A logger is said to be a parent of a child logger if there are no ancestors between itself and the descendant logger.*  
A logger named `com.foo` is parent of logger named `com.foo.Bar`. Similarly, `java` is a parent of `java.util` and an ancestor of `java.util.Vector`.  

The root logger resides at the top of the logger hierarchy. To get reference to root logger:

In [ ]:
Logger rootLogger = LoggerFactory.getLogger(org.slf4j.Logger.ROOT_LOGGER_NAME);

Loggers may be assigned *levels*. The set of possible levels are (in increasing order):
- TRACE
- DEBUG
- INFO
- WARN 
- ERROR

If a given logger is not assigned a level, then it inherits one from its closest ancestor with an assigned level. To ensure that all loggers can eventually inherit a level, the root logger always has an assigned level. By default, this level is DEBUG. For example:
```
------------------------------------------------
Logger name 	Assigned level 	Effective level
------------------------------------------------
root            DEBUG           DEBUG
X               INFO            INFO
X.Y             none            INFO
X.Y.Z           none            INFO
------------------------------------------------
```

Each logger has the following printing methods:

In [ ]:
package org.slf4j; 
public interface Logger {

  // Printing methods: 
  public void trace(String message);
  public void debug(String message);
  public void info(String message); 
  public void warn(String message); 
  public void error(String message); 
}

A logging request is said to be enabled if its level is higher than or equal to the effective level of its logger. Otherwise, the request is said to be disabled. More formally, *A log request of level p issued to a logger having an effective level q, is enabled if p >= q.* 

```
-----------------------------------------------------------
level of                 effective level q
request p    TRACE    DEBUG    INFO    WARN    ERROR    OFF
-----------------------------------------------------------
TRACE        YES      NO       NO      NO      NO       NO
DEBUG        YES      YES      NO      NO      NO       NO
INFO         YES      YES      YES     NO      NO       NO
WARN         YES      YES      YES     YES     NO       NO
ERROR        YES      YES      YES     YES     YES      NO
-----------------------------------------------------------
```

In [ ]:
Logger logger = LoggerFactory.getLogger("com.foo");
//set its Level to INFO. The setLevel() method requires a logback logger
// so the above Logger class should be imported from Logback, not slf4j
logger.setLevel(Level.INFO);

Logger barlogger = LoggerFactory.getLogger("com.foo.Bar");

// This request is enabled, because WARN >= INFO
logger.warn("Low fuel level.");

// This request is disabled, because DEBUG < INFO. 
logger.debug("Starting search for nearest gas station.");

// The logger instance barlogger, named "com.foo.Bar", 
// will inherit its level from the logger named 
// "com.foo" Thus, the following request is enabled 
// because INFO >= INFO. 
barlogger.info("Located nearest gas station.");

// This request is disabled, because DEBUG < INFO. 
barlogger.debug("Exiting gas station search");

**Appenders and Layouts:** In logback speak, an output destination is called an appender. Currently, appenders exist for the console, files, remote socket servers, to MySQL, PostgreSQL, Oracle and other databases, JMS, and remote UNIX Syslog daemons. More than one appender can be attached to a logger. Each enabled logging request for a given logger will be forwarded to all the appenders in that logger as well as the appenders higher in the hierarchy.

*The output of a log statement of logger L will go to all the appenders in L and its ancestors. This is the meaning of the term "appender additivity".  
However, if an ancestor of logger L, say P, has the additivity flag set to false, then L's output will be directed to all the appenders in L and its ancestors up to and including P but not the appenders in any of the ancestors of P.  
Loggers have their additivity flag set to true by default.*

```
-------------------------------------------------------------------------------------------------------------------------
Logger Name    Attached Appenders    Additivity Flag    Output Targets    Comment
-------------------------------------------------------------------------------------------------------------------------
root           A1                    not applicable     A1                Since the root logger stands at the top of 
                                                                          the logger hierarchy, the additivity flag 
                                                                          does not apply to it.
x             A-x1, A-x2             true               A1, A-x1, A-x2    Appenders of "x" and of root.
x.y           none                   true               A1, A-x1, A-x2    Appenders of "x" and of root.
x.y.z         A-xyz                  true               A1, A-x1, A-x2,   Appenders of "x.y.z", "x" and of root.
                                                        A-xyz1
sec           A-sec                  false              A-sec             No appender accumulation since the additivity 
                                                                          flag is set to false. Only appender A-sec 
                                                                          will be used.
sec.access    none                   true               A-sec             Only appenders of "security" because the 
                                                                          additivity flag in "security" is set to false.
-------------------------------------------------------------------------------------------------------------------------                       
```

The output format is controlled via associating a layout with an appender.

### Configuration
Logback can be configured either programmatically or with a configuration script expressed in XML or Groovy format. Logback looks for configuration in the following files in order:
- system property `logback.configurationFile`
- `logback-test.xml` in the classpath
- `logback.xml` in the classpath

Sample configuration file:
```xml
<configuration>
  <appender name="STDOUT" class="ch.qos.logback.core.ConsoleAppender">
    <!-- encoders are assigned the type
         ch.qos.logback.classic.encoder.PatternLayoutEncoder by default -->
    <encoder>
      <pattern>%d{HH:mm:ss.SSS} [%thread] %-5level %logger{36} -%kvp- %msg%n</pattern>
    </encoder>
  </appender>
    
  <logger name="com.foo" level="INFO"/> <!-- additivity is an optional parameter -->

  <root level="debug"> <!-- only level is what we can set for root logger -->
    <appender-ref ref="STDOUT" />
  </root>
</configuration>
```
**Syntax:** 
```
                                       +--- <filter>
                                       +--- <layout>
                                       +--- <encoder>
                    +--- <appender> ---+
<configuration> ----+--- <logger> ---- <appender-ref>
                    +--- <root> --- <appender-ref>
```

We can ask Logback to scan the configuration file and incorporate any configuration change on the fly:
```xml
<configuration scan="true">
```
Scanning is done every minute, we can configure that time:
```xml
<configuration scan="true" scanPeriod="30 seconds" >
```
In case the latest version of the configuration file has XML syntax errors, it will fall back to a previous configuration file free of XML syntax errors.  
We can also show package information in stack trace by:
```xml
<configuration packagingData="true">
```
Sample log:
```
14:28:48.835 [btpool0-7] INFO  c.q.l.demo.prime.PrimeAction - 99 is not a valid value
java.lang.Exception: 99 is invalid
  at ch.qos.logback.demo.prime.PrimeAction.execute(PrimeAction.java:28) [classes/:na]
  at org.apache.struts.action.RequestProcessor.processActionPerform(RequestProcessor.java:431) [struts-1.2.9.jar:1.2.9]
  at org.apache.struts.action.RequestProcessor.process(RequestProcessor.java:236) [struts-1.2.9.jar:1.2.9]
```

By default, appenders are cumulative: a logger will log to the appenders attached to itself (if any) as well as all the appenders attached to its ancestors. Thus, attaching the same appender to multiple loggers will cause logging output to be duplicated.
```xml
<configuration>

  <appender name="STDOUT" class="ch.qos.logback.core.ConsoleAppender">
    <encoder>
      <pattern>%d{HH:mm:ss.SSS} [%thread] %-5level %logger{36} -%kvp- %msg%n</pattern>
    </encoder>
  </appender>

  <!-- Below logger will print the same log twice
       it doesn't need STDOUT appender since it already
       inherits it from the root logger -->  
  <logger name="chapters.configuration">
    <appender-ref ref="STDOUT" />
  </logger>

  <root level="debug">
    <appender-ref ref="STDOUT" />
  </root>
</configuration>
```

**Variable substitution:** Variables can be defined one at a time in the configuration file itself or loaded wholesale from an external properties file or an external resource. For variable aName, the string "${aName}" will be replaced with the value held by the aName variable.
```xml
<variable name="USER_HOME" value="/home/johndoe" />

<appender name="FILE" class="ch.qos.logback.core.FileAppender">
  <file>${USER_HOME}/myApp.log</file>
    <encoder>
      <pattern>%kvp %msg%n</pattern>
  </encoder>
</appender>
```

We can also skip defining the variable in the config, and get it as System property:
```
java -DUSER_HOME="/home/johndoe" MyApp
```

Variables can also be imported from external properties file:
```xml
<variable file="src/main/java/chapters/configuration/variables1.properties" />
```

Default value of a variable can be set (use :-):
```xml
<file>${USER_HOME:-/home/johndoe}/myApp.log</file>
```

**Conditional Processing:** Logback supports conditional processing of configuration files with the help of `<if>`, `<then>` and `<else>` elements so that a single configuration file can adequately target several environments.
```xml
<if condition="some conditional expression">
  <then>
    ...
  </then>
  <else>
    ...
  </else>
</if>
```

Example:
```xml
<configuration debug="true">

  <if condition='property("HOSTNAME").contains("torino")'>
    <then>
      <appender name="CON" class="ch.qos.logback.core.ConsoleAppender">
        <encoder>
          <pattern>%d %-5level %logger{35} -%kvp- %msg %n</pattern>
        </encoder>
      </appender>
      <root>
        <appender-ref ref="CON" />
      </root>
    </then>
  </if>

  <appender name="FILE" class="ch.qos.logback.core.FileAppender">
    <file>${randomOutputDir}/conditional.log</file>
    <encoder>
      <pattern>%d %-5level %logger{35} -%kvp- %msg %n</pattern>
   </encoder>
  </appender>

  <root level="ERROR">
     <appender-ref ref="FILE" />
  </root>
</configuration>
```
Some other functions that we can use include `isDefined` and `isNull`.

**Including config from other file:** this is done by declaring a `<include>` element, as shown:
```xml
<include file="src/main/java/chapters/configuration/includedConfig.xml"/>
```
The file to be included MUST have its elements nested inside an `<included>` element:
```xml
<included>
  <appender name="includedConsole" class="ch.qos.logback.core.ConsoleAppender">
    <encoder>
      <pattern>"%d -%kvp- %m%n"</pattern>
    </encoder>
  </appender>
</included>
```

### Appenders
Appender hierarchy looks like:  
![Appenders](https://logback.qos.ch/manual/images/chapters/appenders/appenderClassDiagram.jpg)

Here we'll focus on `ConsoleAppender` and `FileAppender`s.  
**Console appender:** sample configuration:
```xml
  <appender name="STDOUT" class="ch.qos.logback.core.ConsoleAppender">
    <!-- encoders are assigned the type
         ch.qos.logback.classic.encoder.PatternLayoutEncoder by default -->
    <encoder>
      <pattern>%-4relative [%thread] %-5level %logger{35} - %msg %n</pattern>
    </encoder>
  </appender>
```

**File appender:** appends log events into a file. The target file is specified by the `File` option. If the file already exists, it is either appended to, or truncated depending on the value of the append property. 
```xml
  <appender name="FILE" class="ch.qos.logback.core.FileAppender">
    <file>testFile.log</file>
    <append>true</append> <!-- true is also the default value -->
    <!-- set immediateFlush to false for much higher logging throughput 
         default value of true is safer in the sense that logging events 
         are not lost in case the application exits without properly 
         closing appenders -->
    <immediateFlush>true</immediateFlush>
    <!-- encoders are assigned the type
         ch.qos.logback.classic.encoder.PatternLayoutEncoder by default -->
    <encoder>
      <pattern>%-4relative [%thread] %-5level %logger{35} - %msg%n</pattern>
    </encoder>
  </appender>
```

**Rolling file appender:** has the capability to rollover log files. Once a certain condition is met, it can change its logging target to another file. There are two important subcomponents that interact with `RollingFileAppender`:
- `RollingPolicy`: responsible for undertaking the actions required for a rollover
- `TriggeringPolicy`: determines if and exactly when rollover occurs

To be of any use, a `RollingFileAppender` must have both a `RollingPolicy` and a `TriggeringPolicy` set up. However, if its `RollingPolicy` also implements the `TriggeringPolicy` interface, then only the former needs to be specified explicitly. 

`TimeBasedRollingPolicy` is possibly the most popular rolling policy. It defines a rollover policy based on time, for example by day or by month. `TimeBasedRollingPolicy` assumes the responsibility for rollover as well as for the triggering of said rollover.

Sample:
```xml
 <appender name="FILE" class="ch.qos.logback.core.rolling.RollingFileAppender">
    <file>logFile.log</file>
    <rollingPolicy class="ch.qos.logback.core.rolling.TimeBasedRollingPolicy">
      <!-- daily rollover -->
      <fileNamePattern>logFile.%d{yyyy-MM-dd}.log</fileNamePattern>

      <!-- keep 30 days' worth of history capped at 3GB total size -->
      <maxHistory>30</maxHistory>
      <totalSizeCap>3GB</totalSizeCap>

    </rollingPolicy>

    <encoder>
      <pattern>%-4relative [%thread] %-5level %logger{35} - %msg%n</pattern>
    </encoder>
  </appender>
```

The rollover period is inferred from `fileNamePattern`. In above case, new file will be generated every day. Some more examples:
```
--------------------------------------------------------------------------------------------------------------------------------------
fileNamePattern        Rollover schedule                       Example
--------------------------------------------------------------------------------------------------------------------------------------
logfile.%d.log         Daily rollover (at midnight).           <file> property not set: During November 23rd, 2006, logging output
                       This is same as logfile.%d{yyyy-        will go to the file logfile.2006-11-23.log. At midnight and for the 
                       MM-dd}.log                              rest of the 24th, logging output will be directed to 
                                                               logfile.2006-11-24.log.
                                                               
                                                               <file> property set to logfile.log: During November 23rd, 2006, logging
                                                               output will go to the file logfile.log. At midnight the file will get
                                                               renamed to logfile.2006-11-23.log. A new logfile.log file will be 
                                                               created and for the rest of November 24th logging output will be 
                                                               directed to logfile.log.
                                                               
logfile.%d{yyyy/       Rollover at the beginning              <file> property not set: During month of Oct 2006, logging output
MM}.log                of each month.                         will go to the file logfile.2006/10.log. After midnight of Oct 31 and for
                                                               the rest of Nov, logging output will be directed to logfile.2006/11.log.
                                                               
                                                               <file> property set to logfile.log: During month of Oct 2006, logging
                                                               output will go to the file logfile.log. After midnight of Oct 31 the 
                                                               file will get renamed to logfile.2006/10.log. A new logfile.log file 
                                                               will be created and for the rest of November logging output will be 
                                                               directed to logfile.log.
--------------------------------------------------------------------------------------------------------------------------------------
```

### Encoders
Encoders are responsible for transforming an event into a byte array as well as writing out that byte array into an `OutputStream`. Encoders were introduced in logback version 0.9.19. In previous versions, most appenders relied on a layout to transform an event into a string and write it out using a `java.io.Writer`. In previous versions of logback, users would nest a `PatternLayout` within `FileAppender`. Since logback 0.9.19, `FileAppender` and subclasses expect an encoder and no longer take a layout. 

Layouts, are only able to transform an event into a `String`. Moreover, given that a layout has no control over when events get written out, layouts cannot aggregate events into batches. Contrast this with encoders which not only have total control over the format of the bytes written out, but also control when (and if) those bytes get written out.

At the present time, `PatternLayoutEncoder` is the only really useful encoder. It merely wraps a `PatternLayout` which does most of the work. Thus, it may seem that encoders do not bring much to the table except needless complexity. However, with the advent of new and powerful encoders this impression will change.

The pattern specified can contain literal text and format control expressions called *conversion specifiers*. List:
```
--------------------------------------------------------------------------------------------------------------------------------------
Conversion Word                    Effect
--------------------------------------------------------------------------------------------------------------------------------------
%c{length}                         Displayes name of the logger. Length attribute helps shorten the name. 
%lo{length}
%logger{length}

%C{length}                         Displays the name of the class where the log originated from. Length has similar effect as above.
%class{length}

%d{pattern}                        Outputs the date of the logging event.
%date{pattern}
%d{pattern, timezone}
%date{pattern,timezone}
%d{pattern, timezone,locale}
%date{pattern, timezone, locale}

%F                                 File where the log originated.
%file

%L                                 Outputs the line number from where the logging request was issued.
%line

%m                                 Outputs the application-supplied message associated with the logging event.
$msg
%message

%M                                 Outputs the method name where the logging request was issued.
%method

%p                                 Outputs the level of the logging event.
%le
%level

%t                                 Outputs the name of the thread that generated the logging event. 
%thread

%n                                 Outputs the platform dependent line separator character or characters.

%X{key:-defaultVal}                Outputs the MDC (mapped diagnostic context) associated with the thread that generated the logging
%mdc{key:-defaultVal}              event.
--------------------------------------------------------------------------------------------------------------------------------------
```

**Format modifiers:** 	with the aid of format modifiers it is possible to change the minimum and maximum width and the justifications of each data field. The optional format modifier is placed between the percent sign and the conversion character or word. 